In [1]:
import sys
import glob
sys.path.append("../")

path_dir_test_html = "../test_data/html"
list_fpath_test_html = glob.glob(path_dir_test_html + "/*.html")[:5]

print(list_fpath_test_html)

['../test_data/html\\https%3A%2F%2Fpython.langchain.com%2Fapi_reference%2F_f36a1545.html', '../test_data/html\\https%3A%2F%2Fpython.langchain.com%2Fdocs%2Fcontributing%2F_7821225b.html', '../test_data/html\\https%3A%2F%2Fpython.langchain.com%2Fdocs%2Fhow_to%2Fadd_scores_retriever%2F_675ddc58.html', '../test_data/html\\https%3A%2F%2Fpython.langchain.com%2Fdocs%2Fhow_to%2Fagent_executor%2F_6b55a70d.html', '../test_data/html\\https%3A%2F%2Fpython.langchain.com%2Fdocs%2Fhow_to%2Fassign%2F_b35f6acf.html']


In [2]:
from docmesh.extractor.HTMLFileInspector import HTMLFileInspector
from docmesh.text_split import HTMLContentLoader

html_file_loader = HTMLFileInspector()
html_content_loader = HTMLContentLoader()

url, html = html_file_loader.read_html(list_fpath_test_html[2])
docs_langchain = html_content_loader.load(url, html)

print(len(docs_langchain))

13


In [3]:
from docmesh.text_split.DocumentMerger import DocumentMerger

# 최소 토큰 기준으로 chunk 병합
doc_merger = DocumentMerger(min_tokens=500)
docs_merged = doc_merger.merge_documents(docs_langchain)

print(len(docs_merged))

6


In [4]:
from docmesh.text_split.DocumentChunkSplitter import DocumentChunkSplitter

doc_splitter = DocumentChunkSplitter(chunk_size=1000, chunk_overlap=100)
docs_splitted = doc_splitter.chunkify(docs_merged)

print(len(docs_splitted))

12


In [5]:
from docmesh.embedding import EmbeddingModelFactory
from docmesh.vector_store import VectorStoreFactory

# EmbeddingModelFactory를 통해 임베딩 모델 생성
embedding_model = EmbeddingModelFactory.create_embedding_model(
    provider="openai", model_name="text-embedding-ada-002"
)
# VectorStoreFactory를 통해 벡터 스토어 생성
vector_store = VectorStoreFactory.create_vector_store(
    provider="faiss", embedding_model=embedding_model, path="langchain_doc.faiss", 
)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [6]:
vector_store.add_documents(docs_splitted)

In [7]:
query = vector_store.search("what is rag?")
query[0]

Document(id='3caac93b-2684-4056-9c1d-81ac350dae69', metadata={'source': 'https://python.langchain.com/docs/how_to/add_scores_retriever/', 'split_index': 0}, page_content='!function(){function t(t){document.documentElement.setAttribute("data-theme",t)}var e=function(){try{return new URLSearchParams(window.location.search).get("docusaurus-theme")}catch(t){}}()||function(){try{return window.localStorage.getItem("theme")}catch(t){}}();null!==e?t(e):window.matchMedia("(prefers-color-scheme: dark)").matches?t("dark"):(window.matchMedia("(prefers-color-scheme: light)").matches,t("light"))}(),function(){try{const n=new URLSearchParams(window.location.search).entries();for(var[t,e]of n)if(t.startsWith("docusaurus-data-")){var a=t.replace("docusaurus-data-","data-");document.documentElement.setAttribute(a,e)}}catch(t){}}(),document.documentElement.setAttribute("data-announcement-bar-initially-dismissed",function(){try{return"true"===localStorage.getItem("docusaurus.announcement.dismiss")}catch(t

In [8]:
from docmesh.llm import LLMFactory
from docmesh.config import Config
from docmesh.qa_bot import QAService

In [12]:
# 4. LLM 서비스 생성 (팩토리 사용)
llm = LLMFactory.create_llm_service(
    provider=Config.LLM_PROVIDER,
    model=Config.LLM_MODEL,
    temperature=Config.LLM_TEMPERATURE,
)

retriever = vector_store.vectorstore.as_retriever()

In [13]:
bot = QAService(llm, retriever)
bot.answer_question("langchain이란 무엇입니까")

In [15]:
bot.answer_question("아인슈타인의 생일을 알려주세요")

'I do not know.\n\nSources: N/A'